In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import seaborn as sns
from ionpy.analysis import ResultsLoader
from ese.experiment.experiment.ese_exp import CalibrationExperiment
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE/"

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
train_path = f"{root}/training/10_23_23_Dense_WMH"

dfc = rs.load_configs(
    train_path,
    properties=False,
)

In [ ]:
# Set the device
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

best_exp = rs.get_best_experiment(
    df=rs.load_metrics(dfc),
    exp_class=CalibrationExperiment,
    device="cuda"
)

In [ ]:
best_exp.vis_loss_curves(height=6)

In [ ]:
from ese.experiment.analysis.inference import load_cal_inference_stats

inference_path = f"{root}/inference/11_04_23_WMH_Fixed_Metrics_Inference"

cal_inference_info = load_cal_inference_stats(
    inference_path,
    pixel_stats=False,
    image_stats=True,
    return_metadata=True
)

In [ ]:
cal_inference_info['metadata'].unique_per_col() # Currently doesn't work.

In [ ]:
# Get the pixel preds out
# pixel_preds_dict = pix_inference_dicts[metadata_df['log_set'][0]]

## Pixel-level Analysis

In [ ]:
# from ese.experiment.analysis.err_diagrams import viz_accuracy_vs_confidence

# viz_accuracy_vs_confidence(
#     pixel_preds_dict,
#     title="WMH Confidence vs Accuracy per (Bin and Predicted Label)",
#     x="pred_label",
#     hue="measure",
#     col="bin_num",
#     kind="bar",
#     add_average=True,
#     add_proportion=True,
#     facet_kws={'sharey': False, 'sharex': False}
#     )

In [ ]:
# from ese.experiment.analysis.err_diagrams import viz_accuracy_vs_confidence

# viz_accuracy_vs_confidence(
#     pixel_preds_dict,
#     title="WMH Confidence vs Accuracy per (Bin and Num Neighbors)",
#     x="num_neighbors",
#     hue="measure",
#     col="bin_num",
#     kind="bar",
#     add_average=True,
#     add_proportion=True,
#     facet_kws={'sharey': False, 'sharex': False},
#     )

## Image-level Analysis

In [ ]:
image_info_df = cal_inference_info['image_info_df']

In [ ]:
image_info_df.head()

In [ ]:
# Now using seaborn's FacetGrid to create the KDE plots for the 'accuracy' column for each 'split'.
g = sns.FacetGrid(image_info_df, col="split", col_wrap=3, sharex=True, sharey=True)
g = g.map(sns.kdeplot, "accuracy", fill=True)

# Adjusting the layout
g.fig.tight_layout()

In [ ]:
# Now using seaborn's FacetGrid to create the KDE plots for the 'accuracy' column for each 'split'.
g = sns.FacetGrid(image_info_df, col="split", col_wrap=3, sharex=True, sharey=True)
g = g.map(sns.kdeplot, "dice", fill=True, color="green")

# Adjusting the layout
g.fig.tight_layout()

In [ ]:
from ese.experiment.analysis.err_diagrams import viz_cal_metric_corr

viz_cal_metric_corr(
    image_info_df,
    title="WMH Calibration Metric Correlation",
    col="split"
    )